# Specify Path of MINE_net to load

In [65]:
import os
from os import path as p

proj_path = p.dirname(p.abspath('MINE-Entropy'))
print(proj_path)

/Users/Thomas/Projects/MINE-Entropy


In [66]:
exp_path = p.join(proj_path, "minee/experiments")
print(exp_path)

/Users/Thomas/Projects/MINE-Entropy/minee/experiments


In [67]:
batch_size=2
population_size=512
sub_path = p.join(exp_path, "pop={}_batch={}".format(population_size, batch_size))
print(sub_path)

/Users/Thomas/Projects/MINE-Entropy/minee/experiments/pop=512_batch=2


In [68]:
data_name="Gaussian"
vary_name="rho"
vary_val=0.8
data_path = p.join(sub_path, "{}_{}={}".format(data_name, vary_name, vary_val))
print(data_path)

/Users/Thomas/Projects/MINE-Entropy/minee/experiments/pop=512_batch=2/Gaussian_rho=0.8


In [69]:
model_name="MINE_direct"
model_path = p.join(data_path, model_name)
print(model_path)

/Users/Thomas/Projects/MINE-Entropy/minee/experiments/pop=512_batch=2/Gaussian_rho=0.8/MINE_direct


# Import minee and needed packages

In [70]:
import minee 
import torch

In [71]:
from minee import settings
MINE_dir = settings.model['MINE_direct']['model']

# Load Mine_net

In [72]:
iter_num = 78125
mine_file = p.join(model_path, "checkpoint_iter={}.pt".format(iter_num))
MINE_dir.mine_net.load_state_dict(torch.load(mine_file))

# Specify data parameters

In [74]:
#Default value of datasets are shown
#Data can also be user-specified
def generate_dataset(selected_data):
    if (selected_data == 'Mixed Gaussian'):
        dataset = minee.data.mix_gaussian.MixedGaussian(sample_size=400, mean1=0, mean2=0, rho1=0.9, rho2=-0.9, mix=0.5, theta=0)
    elif (selected_data == 'Gaussian'):
        dataset = minee.data.gaussian.Gaussian(sample_size=200, mean1=0, mean2=1, rho=0.8)
    elif (selected_data == 'Mixed Uniform'):
        dataset = minee.data.mix_uniform.MixedUniform(mix=0.5, width_a=10, width_b=10, sample_size=200)
    elif (selected_data == 'Self-Defined'):
        pass
        #define custom dataset, n by 2 matrix with n being number of samples
        #can also define ground truth
        #dataset = 
    return dataset
gaus = generate_dataset('Gaussian')
X = gaus.data

# Load Train and valid mi_lb curve

In [ ]:
import numpy as np
MINE_dir.avg_train_mi_lb = np.loadtxt(p.join(model_path, "avg_train_mi_lb.txt"))
MINE_dir.avg_valid_mi_lb = np.loadtxt(p.join(model_path, "avg_valid_mi_lb.txt"))
# MINE_dir.avg_train_loss = np.loadtxt(p.join(model_path, "avg_train_loss.txt"))
# MINE_dir.avg_test_loss = np.loadtxt(p.join(model_path, "avg_test_loss.txt"))

# Generate figure

In [83]:
from sklearn.model_selection import train_test_split
MINE_dir.X_train, MINE_dir.X_test = train_test_split(X, test_size=0.35, random_state=0)
MINE_dir.ground_truth = gaus.ground_truth
mi_lb = MINE_dir.forward_pass(MINE_dir.X_test).item()
MINE_dir.savefig(X, mi_lb, suffix="_test2")